# Section 2.2: Quantum Approximate Optimization Algorithm (QAOA) for Max-Cut

## Variational quantum-classical optimization for combinatorial problems

In this section, you'll learn how QAOA tackles hard combinatorial optimization problems by encoding them into quantum Hamiltonians. We'll apply QAOA to the Max-Cut problem on weighted graphs, demonstrating how quantum variational algorithms combine quantum state preparation with classical optimization to find approximate solutions to NP-hard problems.

## Learning Objectives

By the end of this section, you will be able to:

- Understand the Max-Cut problem and its computational complexity
- Represent combinatorial optimization problems as quantum Hamiltonians
- Build QAOA circuits with problem (cost) and mixer Hamiltonians
- Encode graph problems into quantum states using qubit-to-node mapping
- Evaluate cost functions through measurement statistics
- Optimize variational parameters using grid search or gradient-based methods
- Compare QAOA solutions to classical approximation algorithms
- Visualize optimization landscapes and solution quality

In [ ]:
import cirq
import networkx as nx
import sympy
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple
from collections import Counter

## 1. The Max-Cut Problem

Max-Cut is a classic combinatorial optimization problem: given a weighted graph, partition the nodes into two sets to maximize the total weight of edges crossing the partition. This problem is NP-hard, meaning no known polynomial-time algorithm can solve all instances exactly.

In [ ]:
print("Max-Cut Problem Definition:")
print("  Given: Weighted graph G = (V, E) with edge weights w_ij")
print("  Goal: Partition nodes into two sets S and V\\S")
print("  Objective: Maximize sum of weights crossing the partition\n")

print("Example: Consider a simple graph with 3 nodes and weighted edges")
simple_graph = nx.Graph()
simple_graph.add_weighted_edges_from([
    (0, 1, 5.0),
    (1, 2, 3.0),
    (0, 2, 4.0)
])

print(f"  Nodes: {list(simple_graph.nodes())}")
print(f"  Edges with weights:")
for u, v, data in simple_graph.edges(data=True):
    print(f"    ({u}, {v}): weight = {data['weight']:.1f}")

print("\n  Possible partitions:")
partitions = [
    ([0], [1, 2]),
    ([1], [0, 2]),
    ([2], [0, 1]),
    ([0, 1], [2])
]

for s1, s2 in partitions:
    cut_value = 0.0
    for u, v, data in simple_graph.edges(data=True):
        if (u in s1 and v in s2) or (u in s2 and v in s1):
            cut_value += data['weight']
    print(f"    {s1} | {s2}: cut value = {cut_value:.1f}")

print("\n  Maximum cut: {0} | {1,2} or {2} | {0,1} with value 9.0")
print("\nComplexity: NP-hard for general graphs (exponential classical worst case)")

## 2. Creating the Problem Graph

We'll work with a 5-node weighted graph. Each node will map to one qubit, and edge weights determine the relative importance of separating connected nodes into different partitions.

In [ ]:
print("Creating weighted graph for Max-Cut problem...\n")

# Create the graph
graph = nx.Graph()
graph.add_weighted_edges_from([
    (0, 1, 5.0),   # Strong connection
    (0, 3, 2.0),
    (1, 2, 3.0),
    (1, 3, 1.0),
    (2, 3, 4.0),
    (2, 4, 6.0),   # Strong connection
    (3, 4, 2.5)
])

print(f"Graph properties:")
print(f"  Number of nodes: {graph.number_of_nodes()}")
print(f"  Number of edges: {graph.number_of_edges()}")
print(f"  Total edge weight: {sum(data['weight'] for _, _, data in graph.edges(data=True)):.1f}")

print(f"\nEdges and weights:")
for u, v, data in graph.edges(data=True):
    print(f"  Edge ({u}, {v}): weight = {data['weight']:.1f}")

print("\nNote: Higher weights indicate edges we prefer to cut")
print("      Optimal solution should separate nodes connected by heavy edges")

### 2.1 Visualizing the Problem Graph

Visual representation helps understand the structure and identify potential good partitions by inspection.

In [ ]:
print("Visualizing the problem graph...\n")

plt.figure(figsize=(10, 8))

# Layout for visualization
pos = nx.spring_layout(graph, seed=42)

# Draw nodes
nx.draw_networkx_nodes(graph, pos, node_color='lightblue', 
                      node_size=800, alpha=0.9)
nx.draw_networkx_labels(graph, pos, font_size=16, 
                       font_weight='bold')

# Draw edges with width proportional to weight
weights = [data['weight'] for _, _, data in graph.edges(data=True)]
max_weight = max(weights)

for u, v, data in graph.edges(data=True):
    weight = data['weight']
    width = 1 + 4 * (weight / max_weight)  # Scale width by weight
    nx.draw_networkx_edges(graph, pos, [(u, v)], width=width, 
                          alpha=0.6, edge_color='gray')

# Draw edge labels
edge_labels = {(u, v): f"{data['weight']:.1f}" 
               for u, v, data in graph.edges(data=True)}
nx.draw_networkx_edge_labels(graph, pos, edge_labels, font_size=10)

plt.title("Max-Cut Problem Graph\n(Edge thickness proportional to weight)", 
         fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.savefig('/Users/tylerhayden/Projects/demos/cirq-101/notebooks/qaoa_graph.png', 
            dpi=150, bbox_inches='tight')
print("Saved graph visualization to: qaoa_graph.png")
plt.show()

print("\nObservation: Thick edges (high weights) are good candidates for cutting")

## 3. QAOA Algorithm Overview

QAOA is a variational quantum-classical hybrid algorithm:

1. **Quantum Step**: Prepare parameterized quantum state |ψ(γ,β)⟩
2. **Measurement**: Sample bitstrings from the quantum state
3. **Classical Evaluation**: Calculate average cost from samples
4. **Classical Optimization**: Update parameters (γ,β) to minimize cost
5. **Iterate**: Repeat until convergence

The quantum state encodes candidate solutions, and the variational principle drives the system toward high-quality solutions.

In [ ]:
print("QAOA Algorithm Structure:\n")

print("1. INITIALIZATION")
print("   - Apply Hadamard gates to all qubits")
print("   - Creates uniform superposition over all 2^n bitstrings")
print("   - |ψ₀⟩ = H^⊗n|0⟩^⊗n = (1/√2^n) Σ|x⟩\n")

print("2. PROBLEM HAMILTONIAN (Cost Layer)")
print("   - Encode Max-Cut into quantum Hamiltonian H_C")
print("   - H_C = Σ_{(i,j)∈E} w_ij (I - Z_i Z_j) / 2")
print("   - Apply evolution: exp(-i γ H_C)")
print("   - Physics: ZZ interactions encode edge weights\n")

print("3. MIXER HAMILTONIAN (Exploration Layer)")
print("   - H_M = Σ_i X_i (sum of X operators)")
print("   - Apply evolution: exp(-i β H_M)")
print("   - Physics: Explores different partitions\n")

print("4. MEASUREMENT AND EVALUATION")
print("   - Measure all qubits in computational basis")
print("   - Each bitstring represents a partition")
print("   - Calculate cut value for each measured partition")
print("   - Average gives expectation value ⟨H_C⟩\n")

print("5. CLASSICAL OPTIMIZATION")
print("   - Objective: Maximize average cut value")
print("   - Optimize parameters (γ, β) using classical optimizer")
print("   - Methods: Grid search, gradient descent, evolutionary algorithms\n")

print("Key Insight:")
print("  - γ controls problem-specific phase")
print("  - β controls exploration/mixing")
print("  - Balance needed for optimal performance")

## 4. QAOA Circuit Structure

The QAOA ansatz circuit consists of alternating problem and mixer layers. For single-layer QAOA (p=1), we have one application of each. Deeper circuits (p>1) repeat these layers with independent parameters.

In [ ]:
print("Building QAOA circuit with symbolic parameters...\n")

# Define symbolic parameters
gamma = sympy.Symbol('gamma')
beta = sympy.Symbol('beta')

print(f"Symbolic parameters:")
print(f"  γ (gamma): {gamma}")
print(f"  β (beta): {beta}")

# Map nodes to qubits
qubits = sorted([cirq.LineQubit(i) for i in graph.nodes()])
print(f"\nQubit mapping:")
for i, q in enumerate(qubits):
    print(f"  Node {i} → Qubit {q}")

# Build circuit
circuit = cirq.Circuit()

# Step 1: Initialize superposition
circuit.append(cirq.H.on_each(*qubits))

print(f"\nCircuit structure:")
print(f"  1. Initialization: Hadamard on all {len(qubits)} qubits")

# Step 2: Cost layer (problem Hamiltonian)
print(f"\n  2. Cost layer (Problem Hamiltonian):")
cost_gates = 0
for u, v, data in graph.edges(data=True):
    weight = data['weight']
    # Implement exp(-i * gamma * w_ij * (I - Z_i Z_j) / 2)
    # Decomposes to: CNOT(u,v), RZ(2*gamma*weight, v), CNOT(u,v)
    circuit.append([
        cirq.CNOT(qubits[u], qubits[v]),
        cirq.rz(2 * gamma * weight).on(qubits[v]),
        cirq.CNOT(qubits[u], qubits[v])
    ])
    cost_gates += 1
    print(f"     Edge ({u},{v}), weight={weight:.1f}: CNOT-RZ(2γ·{weight:.1f})-CNOT")

# Step 3: Mixer layer
print(f"\n  3. Mixer layer (Exploration Hamiltonian):")
print(f"     RX(-2β) on all {len(qubits)} qubits")
circuit.append(cirq.rx(-2 * beta).on_each(*qubits))

# Add measurements
circuit.append(cirq.measure(*qubits, key='result'))

print(f"\n  4. Measurement: All qubits in computational basis")
print(f"\nTotal circuit depth: ~{len(circuit)} moments")
print(f"Total gates: {sum(len(moment) for moment in circuit)} gates\n")

print("Circuit diagram (first few moments):")
print(circuit)

### 4.1 Understanding the Cost Layer Decomposition

The cost layer implements exp(-i γ H_C) where H_C encodes the Max-Cut problem. Each edge contributes a ZZ interaction term.

In [ ]:
print("Cost Layer Physics:\n")

print("Hamiltonian for single edge (u,v) with weight w:")
print("  H_edge = w (I - Z_u Z_v) / 2\n")

print("Why this encoding?")
print("  - If qubits u,v in SAME partition: Z_u Z_v = +1")
print("    → (I - Z_u Z_v)/2 = (1-1)/2 = 0  (no contribution)")
print("  - If qubits u,v in DIFFERENT partitions: Z_u Z_v = -1")
print("    → (I - Z_u Z_v)/2 = (1-(-1))/2 = 1  (contributes weight w)\n")

print("Total Hamiltonian:")
print("  H_C = Σ_{edges} w_ij (I - Z_i Z_j) / 2")
print("  Ground state of -H_C → Maximum cut\n")

print("Gate decomposition for exp(-i γ w (I - Z_u Z_v) / 2):")
print("  1. CNOT(u, v)     - Entangle qubits")
print("  2. RZ(2γw, v)     - Apply phase based on edge weight")
print("  3. CNOT(u, v)     - Disentangle qubits\n")

print("Example: For edge (0,1) with weight 5.0:")
q0, q1 = cirq.LineQubit(0), cirq.LineQubit(1)
gamma_val = sympy.Symbol('gamma')
edge_circuit = cirq.Circuit([
    cirq.CNOT(q0, q1),
    cirq.rz(2 * gamma_val * 5.0).on(q1),
    cirq.CNOT(q0, q1)
])
print(edge_circuit)

print("\nThis decomposition is exact and preserves unitarity")

## 5. Encoding Max-Cut as a Quantum Hamiltonian

The mapping from classical optimization to quantum Hamiltonian is crucial. We encode partitions as computational basis states and the cut value as energy.

In [ ]:
print("Classical to Quantum Encoding:\n")

print("1. PARTITION REPRESENTATION")
print("   Classical: Partition S ⊆ V (set of nodes)")
print("   Quantum: Bitstring |x⟩ where x_i ∈ {0,1}")
print("   Example: |01101⟩ means nodes {1,2,4} in partition 1\n")

print("2. CUT VALUE CALCULATION")
print("   Classical: Cut(S) = Σ_{i∈S, j∉S} w_ij")
print("   Quantum: ⟨x|H_C|x⟩ = Σ_{(i,j)∈E} w_ij (1 - x_i x_j) / 2")
print("   where x_i ∈ {-1,+1} (eigenvalues of Z)\n")

print("3. EXAMPLE: Calculate cut value for partition |01010⟩")
example_partition = [0, 1, 0, 1, 0]
print(f"   Bitstring: {example_partition}")
print(f"   Partition: {{1, 3}} vs {{0, 2, 4}}\n")

cut_value = 0.0
print("   Edge contributions:")
for u, v, data in graph.edges(data=True):
    weight = data['weight']
    if example_partition[u] != example_partition[v]:
        cut_value += weight
        print(f"     ({u},{v}): nodes in different partitions → +{weight:.1f}")
    else:
        print(f"     ({u},{v}): nodes in same partition → +0.0")

print(f"\n   Total cut value: {cut_value:.1f}")

print("\n4. OPTIMIZATION OBJECTIVE")
print("   Classical: Find partition maximizing Cut(S)")
print("   Quantum: Find state |ψ⟩ maximizing ⟨ψ|H_C|ψ⟩")
print("   QAOA achieves this through variational optimization")

## 6. Cost Function Evaluation

The quantum circuit produces measurement samples. We evaluate the cost by calculating the cut value for each measured bitstring and averaging.

In [ ]:
print("Evaluating QAOA cost function with sample parameters...\n")

# Use sample parameters
gamma_test = 0.5
beta_test = 0.3

print(f"Test parameters: γ = {gamma_test:.2f}, β = {beta_test:.2f}\n")

# Build circuit (without symbolic parameters this time)
test_qubits = sorted([cirq.LineQubit(i) for i in graph.nodes()])
test_circuit = cirq.Circuit()

# Initialize
test_circuit.append(cirq.H.on_each(*test_qubits))

# Cost layer with numerical values
for u, v, data in graph.edges(data=True):
    weight = data['weight']
    test_circuit.append([
        cirq.CNOT(test_qubits[u], test_qubits[v]),
        cirq.rz(2 * gamma_test * weight).on(test_qubits[v]),
        cirq.CNOT(test_qubits[u], test_qubits[v])
    ])

# Mixer layer
test_circuit.append(cirq.rx(-2 * beta_test).on_each(*test_qubits))

# Measurement
test_circuit.append(cirq.measure(*test_qubits, key='result'))

# Run circuit
simulator = cirq.Simulator()
repetitions = 1000
print(f"Running circuit with {repetitions} measurements...\n")

results = simulator.run(test_circuit, repetitions=repetitions)
measurements = results.measurements['result']

# Calculate cut values
cut_values = []
for sample in measurements:
    cut = 0.0
    for u, v, data in graph.edges(data=True):
        if sample[u] != sample[v]:
            cut += data['weight']
    cut_values.append(cut)

# Statistics
avg_cut = np.mean(cut_values)
std_cut = np.std(cut_values)
max_cut = np.max(cut_values)
min_cut = np.min(cut_values)

print(f"Results from {repetitions} samples:")
print(f"  Average cut value: {avg_cut:.2f}")
print(f"  Standard deviation: {std_cut:.2f}")
print(f"  Maximum observed: {max_cut:.2f}")
print(f"  Minimum observed: {min_cut:.2f}")

# Show distribution
print(f"\nCut value distribution:")
unique_cuts, counts = np.unique(cut_values, return_counts=True)
for cut_val, count in sorted(zip(unique_cuts, counts), reverse=True):
    percentage = 100 * count / repetitions
    bar = '█' * int(percentage / 2)
    print(f"  {cut_val:5.1f}: {count:4d} ({percentage:5.1f}%) {bar}")

print(f"\nNote: Average cut value is the QAOA objective function")
print(f"      Higher values indicate better solutions")

### 6.1 Understanding Measurement Statistics

The distribution of measured bitstrings reveals how the quantum state concentrates probability on good solutions.

In [ ]:
print("Analyzing measurement bitstrings...\n")

# Get histogram of bitstrings
histogram = results.histogram(key='result')

print(f"Top 10 most frequently measured bitstrings:\n")
print(f"  Rank | Bitstring | Count | Frequency | Cut Value")
print(f"  " + "-" * 55)

for rank, (bitstring_int, count) in enumerate(histogram.most_common(10), 1):
    # Convert integer to bitstring
    bitstring = [int(b) for b in format(bitstring_int, f'0{len(test_qubits)}b')]
    
    # Calculate cut value
    cut = 0.0
    for u, v, data in graph.edges(data=True):
        if bitstring[u] != bitstring[v]:
            cut += data['weight']
    
    frequency = count / repetitions
    bitstring_str = ''.join(map(str, bitstring))
    
    print(f"  {rank:4d} | {bitstring_str} |  {count:4d} |   {frequency:.3f}  |   {cut:.1f}")

print("\nObservations:")
print("  - Good solutions (high cut values) should appear frequently")
print("  - Poor solutions (low cut values) should be suppressed")
print("  - Parameter optimization improves this concentration")

## 7. Optimization and Parameter Tuning

Finding optimal (γ, β) parameters is crucial for QAOA performance. We'll use grid search to visualize the optimization landscape and find good parameters.

In [ ]:
print("Optimizing QAOA parameters via grid search...\n")

# Define parameter grid
# For single-layer QAOA, typical ranges are γ ∈ [0, π] and β ∈ [0, π/2]
gamma_range = np.linspace(0, np.pi, 15)
beta_range = np.linspace(0, np.pi/2, 15)

print(f"Parameter ranges:")
print(f"  γ (gamma): {gamma_range[0]:.3f} to {gamma_range[-1]:.3f} ({len(gamma_range)} points)")
print(f"  β (beta):  {beta_range[0]:.3f} to {beta_range[-1]:.3f} ({len(beta_range)} points)")
print(f"  Total evaluations: {len(gamma_range) * len(beta_range)}\n")

print("Note: Using reduced repetitions (500) for faster grid search")
print("      Full optimization would use 5000+ repetitions\n")

# Helper function to evaluate cost
def evaluate_qaoa(gamma_val, beta_val, reps=500):
    """Evaluate average cut value for given parameters."""
    eval_circuit = cirq.Circuit()
    eval_circuit.append(cirq.H.on_each(*test_qubits))
    
    # Cost layer
    for u, v, data in graph.edges(data=True):
        weight = data['weight']
        eval_circuit.append([
            cirq.CNOT(test_qubits[u], test_qubits[v]),
            cirq.rz(2 * gamma_val * weight).on(test_qubits[v]),
            cirq.CNOT(test_qubits[u], test_qubits[v])
        ])
    
    # Mixer layer
    eval_circuit.append(cirq.rx(-2 * beta_val).on_each(*test_qubits))
    eval_circuit.append(cirq.measure(*test_qubits, key='result'))
    
    # Run and evaluate
    results = simulator.run(eval_circuit, repetitions=reps)
    measurements = results.measurements['result']
    
    total_cut = 0.0
    for sample in measurements:
        for u, v, data in graph.edges(data=True):
            if sample[u] != sample[v]:
                total_cut += data['weight']
    
    return total_cut / reps

# Perform grid search (this takes a minute)
print("Running grid search...")
cost_grid = np.zeros((len(gamma_range), len(beta_range)))

for i, gamma_val in enumerate(gamma_range):
    for j, beta_val in enumerate(beta_range):
        cost_grid[i, j] = evaluate_qaoa(gamma_val, beta_val)
    
    if (i + 1) % 3 == 0:
        print(f"  Progress: {i+1}/{len(gamma_range)} gamma values completed")

# Find optimal parameters
best_idx = np.unravel_index(np.argmax(cost_grid), cost_grid.shape)
best_gamma = gamma_range[best_idx[0]]
best_beta = beta_range[best_idx[1]]
best_cost = cost_grid[best_idx]

print(f"\nOptimization complete!")
print(f"  Best parameters found:")
print(f"    γ = {best_gamma:.4f}")
print(f"    β = {best_beta:.4f}")
print(f"  Best average cut value: {best_cost:.2f}")

### 7.1 Visualizing the Optimization Landscape

The optimization landscape shows how the average cut value varies with (γ, β). This reveals the structure of the variational problem.

In [ ]:
print("Creating optimization landscape visualization...\n")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Create meshgrid
Gamma, Beta = np.meshgrid(gamma_range, beta_range, indexing='ij')

# 1. Contour plot
levels = 15
contour = ax1.contourf(Gamma, Beta, cost_grid, levels=levels, cmap='viridis')
fig.colorbar(contour, ax=ax1, label='Average Cut Value')

# Overlay contour lines
ax1.contour(Gamma, Beta, cost_grid, levels=levels, colors='white', 
           alpha=0.3, linewidths=0.5)

# Mark optimal point
ax1.plot(best_gamma, best_beta, 'r*', markersize=20, 
        label=f'Optimum (γ={best_gamma:.3f}, β={best_beta:.3f})')

ax1.set_xlabel('γ (Gamma)', fontsize=12)
ax1.set_ylabel('β (Beta)', fontsize=12)
ax1.set_title('QAOA Optimization Landscape', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# 2. 3D surface plot
ax2 = fig.add_subplot(122, projection='3d')
surf = ax2.plot_surface(Gamma, Beta, cost_grid, cmap='viridis', 
                       alpha=0.8, edgecolor='none')
ax2.scatter([best_gamma], [best_beta], [best_cost], 
           color='red', s=100, marker='*', label='Optimum')

ax2.set_xlabel('γ (Gamma)', fontsize=11)
ax2.set_ylabel('β (Beta)', fontsize=11)
ax2.set_zlabel('Avg Cut Value', fontsize=11)
ax2.set_title('3D Optimization Surface', fontsize=14, fontweight='bold')
ax2.view_init(elev=25, azim=45)

plt.tight_layout()
plt.savefig('/Users/tylerhayden/Projects/demos/cirq-101/notebooks/qaoa_landscape.png', 
            dpi=150, bbox_inches='tight')
print("Saved optimization landscape to: qaoa_landscape.png")
plt.show()

print("\nLandscape characteristics:")
print(f"  Global maximum: {np.max(cost_grid):.2f}")
print(f"  Global minimum: {np.min(cost_grid):.2f}")
print(f"  Range: {np.max(cost_grid) - np.min(cost_grid):.2f}")
print(f"\n  Note: Smooth landscape suggests gradient-based methods would work well")

## 8. Extracting and Analyzing the Solution

With optimized parameters, we extract the final solution and compare it to classical approaches.

In [ ]:
print("Extracting QAOA solution with optimized parameters...\n")

# Run with optimal parameters and many samples
repetitions_final = 5000
print(f"Running optimized circuit with {repetitions_final} measurements...\n")

final_circuit = cirq.Circuit()
final_circuit.append(cirq.H.on_each(*test_qubits))

# Cost layer with optimal gamma
for u, v, data in graph.edges(data=True):
    weight = data['weight']
    final_circuit.append([
        cirq.CNOT(test_qubits[u], test_qubits[v]),
        cirq.rz(2 * best_gamma * weight).on(test_qubits[v]),
        cirq.CNOT(test_qubits[u], test_qubits[v])
    ])

# Mixer layer with optimal beta
final_circuit.append(cirq.rx(-2 * best_beta).on_each(*test_qubits))
final_circuit.append(cirq.measure(*test_qubits, key='result'))

# Execute
final_results = simulator.run(final_circuit, repetitions=repetitions_final)
final_histogram = final_results.histogram(key='result')

# Get most common bitstring
most_common_int, count = final_histogram.most_common(1)[0]
qaoa_partition = [int(b) for b in format(most_common_int, f'0{len(test_qubits)}b')]

# Calculate QAOA cut value
qaoa_cut = 0.0
for u, v, data in graph.edges(data=True):
    if qaoa_partition[u] != qaoa_partition[v]:
        qaoa_cut += data['weight']

print(f"QAOA Solution:")
print(f"  Most frequent bitstring: {''.join(map(str, qaoa_partition))}")
print(f"  Measurement frequency: {count}/{repetitions_final} ({100*count/repetitions_final:.1f}%)")
print(f"  Cut value: {qaoa_cut:.2f}")

# Show partition sets
partition_0 = [i for i, bit in enumerate(qaoa_partition) if bit == 0]
partition_1 = [i for i, bit in enumerate(qaoa_partition) if bit == 1]
print(f"\n  Partition: {{{', '.join(map(str, partition_0))}}} | {{{', '.join(map(str, partition_1))}}}")

# List cut edges
print(f"\n  Edges in cut:")
for u, v, data in graph.edges(data=True):
    if qaoa_partition[u] != qaoa_partition[v]:
        print(f"    ({u}, {v}): weight {data['weight']:.1f}")

total_weight = sum(data['weight'] for _, _, data in graph.edges(data=True))
print(f"\n  Cut percentage: {100 * qaoa_cut / total_weight:.1f}% of total edge weight")

### 8.1 Classical Comparison: Greedy and Random Algorithms

To evaluate QAOA performance, we compare against classical heuristics.

In [ ]:
print("Comparing QAOA to classical algorithms...\n")

# 1. Random partitioning
print("1. RANDOM PARTITIONING")
print("   Strategy: Try 100 random partitions, keep the best\n")

np.random.seed(42)  # For reproducibility
best_random_cut = 0.0
best_random_partition = None

for trial in range(100):
    random_partition = np.random.randint(0, 2, len(graph.nodes()))
    random_cut = 0.0
    for u, v, data in graph.edges(data=True):
        if random_partition[u] != random_partition[v]:
            random_cut += data['weight']
    
    if random_cut > best_random_cut:
        best_random_cut = random_cut
        best_random_partition = random_partition

print(f"   Best random cut value: {best_random_cut:.2f}")
print(f"   Best random partition: {''.join(map(str, best_random_partition))}")

# 2. Greedy algorithm
print("\n2. GREEDY ALGORITHM")
print("   Strategy: For each node, choose partition that maximizes cut\n")

greedy_partition = np.zeros(len(graph.nodes()), dtype=int)
for node in graph.nodes():
    # Try partition 0
    greedy_partition[node] = 0
    cut_0 = 0.0
    for u, v, data in graph.edges(data=True):
        if greedy_partition[u] != greedy_partition[v]:
            cut_0 += data['weight']
    
    # Try partition 1
    greedy_partition[node] = 1
    cut_1 = 0.0
    for u, v, data in graph.edges(data=True):
        if greedy_partition[u] != greedy_partition[v]:
            cut_1 += data['weight']
    
    # Keep better choice
    greedy_partition[node] = 1 if cut_1 > cut_0 else 0

greedy_cut = 0.0
for u, v, data in graph.edges(data=True):
    if greedy_partition[u] != greedy_partition[v]:
        greedy_cut += data['weight']

print(f"   Greedy cut value: {greedy_cut:.2f}")
print(f"   Greedy partition: {''.join(map(str, greedy_partition))}")

# 3. Comparison table
print("\n" + "="*60)
print("ALGORITHM COMPARISON")
print("="*60)
print(f"Algorithm           | Cut Value | Approximation Ratio")
print("-" * 60)

max_cut_found = max(qaoa_cut, best_random_cut, greedy_cut)

print(f"QAOA (quantum)      |   {qaoa_cut:5.2f}   |      {qaoa_cut/max_cut_found:.3f}")
print(f"Greedy (classical)  |   {greedy_cut:5.2f}   |      {greedy_cut/max_cut_found:.3f}")
print(f"Random (classical)  |   {best_random_cut:5.2f}   |      {best_random_cut/max_cut_found:.3f}")
print("="*60)

if qaoa_cut >= max_cut_found:
    print("\nQAOA found the best solution!")
elif qaoa_cut >= greedy_cut:
    print("\nQAOA matched or beat the greedy algorithm")
else:
    print("\nGreedy algorithm found a better solution")
    print("Note: Single-layer QAOA may need deeper circuits for this graph")

### 8.2 Visualizing the Final Solution

Visualize the graph with the QAOA partition, highlighting cut edges.

In [ ]:
print("Visualizing QAOA solution...\n")

# Set up colors based on partition
node_colors = ['red' if bit == 0 else 'blue' for bit in qaoa_partition]

plt.figure(figsize=(12, 9))

# Layout (same as before for consistency)
pos = nx.spring_layout(graph, seed=42)

# Draw nodes with partition colors
nx.draw_networkx_nodes(graph, pos, node_color=node_colors, 
                      node_size=1000, alpha=0.9, edgecolors='black', linewidths=2)
nx.draw_networkx_labels(graph, pos, font_size=16, 
                       font_color='white', font_weight='bold')

# Draw edges - highlight cut edges
for u, v, data in graph.edges(data=True):
    weight = data['weight']
    if qaoa_partition[u] != qaoa_partition[v]:
        # Cut edge - draw thick and green
        nx.draw_networkx_edges(graph, pos, [(u, v)], 
                             width=4, edge_color='green', alpha=0.9)
    else:
        # Non-cut edge - draw thin and gray
        nx.draw_networkx_edges(graph, pos, [(u, v)], 
                             width=1.5, edge_color='gray', alpha=0.4)

# Draw edge labels
edge_labels = {(u, v): f"{data['weight']:.1f}" 
               for u, v, data in graph.edges(data=True)}
nx.draw_networkx_edge_labels(graph, pos, edge_labels, font_size=11)

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='red', label='Partition 0'),
    Patch(facecolor='blue', label='Partition 1'),
    Patch(facecolor='green', label='Cut Edge', alpha=0.9),
    Patch(facecolor='gray', label='Non-cut Edge', alpha=0.4)
]
plt.legend(handles=legend_elements, loc='upper right', fontsize=11)

plt.title(f"QAOA Max-Cut Solution\nCut Value: {qaoa_cut:.2f} (γ={best_gamma:.3f}, β={best_beta:.3f})", 
         fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.savefig('/Users/tylerhayden/Projects/demos/cirq-101/notebooks/qaoa_solution.png', 
            dpi=150, bbox_inches='tight')
print("Saved solution visualization to: qaoa_solution.png")
plt.show()

print(f"\nGreen edges (cut): {sum(1 for u, v, _ in graph.edges(data=True) if qaoa_partition[u] != qaoa_partition[v])}")
print(f"Gray edges (same partition): {sum(1 for u, v, _ in graph.edges(data=True) if qaoa_partition[u] == qaoa_partition[v])}")

## Exercises

1. **Parameter Sensitivity**: Run QAOA with γ = 0.1, β = 0.1 and γ = π, β = π/2. How does performance change? Why?

2. **Different Graphs**: Create a complete graph K₄ (all nodes connected). What is the maximum possible cut? Does QAOA find it?

3. **Depth Exploration**: Modify the circuit to use p=2 layers (two cost + two mixer layers with independent parameters). Does this improve the solution?

4. **Hamiltonian Verification**: Manually calculate ⟨ψ|H_C|ψ⟩ for the all-zeros state |00000⟩. Does it match the cut value?

5. **Optimization Methods**: Implement gradient-free optimization (e.g., Nelder-Mead) instead of grid search. Does it find better parameters faster?

6. **Approximation Ratio**: For random graphs, the Goemans-Williamson algorithm guarantees a 0.878 approximation ratio. How does QAOA compare?

7. **Noise Effects**: Use `cirq.DensityMatrixSimulator` with depolarizing noise. How does solution quality degrade with increasing noise?

## Key Takeaways

- **QAOA is a hybrid quantum-classical algorithm** combining quantum state preparation with classical optimization
- **Max-Cut is NP-hard** but has practical applications in network design, circuit layout, and machine learning
- **Problem encoding uses Hamiltonians** where computational basis states represent candidate solutions
- **Cost Hamiltonian H_C** encodes the optimization objective through ZZ interaction terms
- **Mixer Hamiltonian H_M** enables exploration of the solution space via X rotations
- **Parameters (γ, β) control the balance** between problem-specific structure and solution exploration
- **Measurement statistics provide cost estimates** by sampling the prepared quantum state
- **Grid search reveals optimization landscape** but gradient-based methods scale better
- **Single-layer QAOA (p=1) provides good approximations** but deeper circuits may improve performance
- **QAOA performance depends on problem structure** and can match or exceed classical heuristics
- **Real quantum hardware requires noise mitigation** and error-aware compilation

---

**Next:** Section 2.3 - Quantum Machine Learning - Learn how quantum circuits can be trained for classification and regression tasks